In [5]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

# ランダムな交換により現れる分布について

ランダムな交換や分配によって現れる分布について、プログラムによるシミュレーションを行った。

シミュレーションの中にはかなり時間がかかるものもある。設定されているループの繰り返しの回数は、表示されている結果を得るために必要な回数である。

分布がどのように変わっていくのかを見たい場合は、頭にある変数の初期化部分からループ以降の部分を切り離し、繰り返しの回数を少なくして少しずつやっていくことをお勧めする。

きっかけとなったのは板尾健司氏による以下の記事。

[ランダムな世界では「所得の平等」は実現しない…数理モデルが解き明かした、超富裕層が生まれる仕組み](https://gendai.media/articles/-/148324)


## ランダムな交換（マイナスを許す）

みな同じだけの財産を持っている状態から始めて、ランダムに選んだ二人が一定量の財を交換する（一方が他方に贈与する）。
与える者として多く選ばれると、財産はマイナスになることもある。

結果は正規分布になる。
<img src='image/dist1.png'>

In [171]:
def normal(x, u, v):
    ret = 1 / np.sqrt(2 * np.pi * v) * np.exp(-(x-u)**2/(2*v))
    return ret

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(50000000):
    # ランダムに二人を選ぶ
    x = random.sample(range(N), k=2)
    w[x[0]] -= 1 # 一方に 1 を足す
    w[x[1]] += 1 # 他方から 1 を引く
    num_iter += 1

plt.hist(w, bins=100)
x = np.arange(-200, 401, 10)
plt.plot(x, 74000 * normal(x, 100, 10000), label='正規分布')
plt.xlim(-200, 400)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察
一人の人に注目すると、これは結局「同確率で +1 か -1 する」という試行を繰り返しているにすぎない。よってその分布は二項分布となり、試行の回数を増やせば正規分布に近づいていく。


## ランダムな交換（多い方に１を足し、少ない方から１を引く）

ランダムに二人を選ぶことは同じで、現在の財産が少ないほうが多い方に贈与する。

結果は一様分布になる。
<img src='image/dist8.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(10000000):
    # ランダムに x, y を選ぶ。
    x, y = tuple(random.sample(range(N), k=2))
    # w[x] > w[y]　なら x と y を入れ替える
    if w[x] > w[y]:
        x, y = y, x
    w[x] -= 1 # 少ない方から１を引く
    w[y] += 1 # 多い方に１を足す
    num_iter += 1

plt.hist(w, bins=100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

この結果は一見したところ直感に反するかもしれない。分布の台形の両端はますます左右に離れてゆき、中央あたりは財産の増減が打ち消し合うから、中央が盛り上がった曲線になるのではないかと予想するかもしれない。だが、次のように考えれば分布が水平な台形型になることが納得できるかと思う。

まず、ある時点で分布が台地型になっていると仮定する。

一回の交換で、ある量の財産を持つ人が 1 増える確率は、台地の左端だとゼロ、右端だと 1 で、その間は距離に**比例して**変わっていく。

1 減る確率は、増える場合の左右を逆にしたものとなる。

つまり、ある財産に注目した時、その人数が減る確率=(1増えて右へ移動する確率)+(1減って左へ移動する確率)となり、これは財産によらず一定である。

一方、人数が増える確率=(財産が1多い人が1減らす確率)+(財産が1少ない人が1増やす確率)となり、これも財産にはよらない一定数となる。

つまり、ある財産を持つ人の増減の期待値は財産によらず一定となる。従って分布の台地形は保たれたままとなる。

ただ、台地の高さは低くなってゆき、分布の幅は左右に広がっていく。


## ランダムな交換（ゼロの人からは取らない）

ランダムに選んだ二人が一定量の財を交換するが、ただし財産がゼロの人からは取らない（交換を中止し、選択をやり直す）。
このため財産はゼロより少なくなることはない。

結果は指数分布になる。
<img src='image/dist2.png'>

こちらは y 軸を対数目盛にしたもの。
<img src='image/dist2_2.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(20000000):
    # ランダムに x, y を選ぶ。
    # ただし x の財産がゼロならやり直す。
    x, y = tuple(random.sample(range(N), k=2))
    while w[x] == 0:
        x, y = tuple(random.sample(range(N), k=2))
    w[x] -= 1
    w[y] += 1
    num_iter += 1

print(f'繰り返し数: {num_iter}')

plt.hist(w, bins=790)
x = range(0, 601, 10)
plt.plot(x, 95 * np.power(0.9902, x), label='y=0.9902^x')
plt.xlim(0, 600)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

plt.hist(w, bins=790)
plt.yscale("log")
x = range(0, 601, 10)
plt.plot(x, 95 * np.power(0.9902, x), label='y=0.9902^x')
plt.xlim(0, 600)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

### 考察

「ランダムな交換（マイナスを許す）」に「財産ゼロのひとからは取らない」という条件を加えただけで、結果の分布は正規分布から指数分布へと変化する。

この結果については、厳密ではないが、以下のように考えてみることができる。

人の数を $N$、財産が $x$ の人の人数を $ f(x) $ とすると、一回の試行による $ f(x) $ の増分は、

$x >= 1$ については

$$
\frac{1}{N}f(x-1) + \frac{1}{N-f(0)}f(x+1) - \frac{1}{N}f(x) - \frac{1}{N-f(0)}f(x) \\
= (\frac{1}{N-f(0)}f(x+1) - \frac{1}{N}f(x)) - (\frac{1}{N-f(0)}f(x) - \frac{1}{N}f(x-1))
$$

$x = 0$ については

$$ \frac{1}{N-f(0)}f(1) - \frac{1}{N}f(0) $$

となる。

ここで、試しに $ f(x) = f(0) \cdot c^x $ と置くと、これらの増分は

$$ \frac{c}{N-f(0)} - \frac{1}{N} = 0 $$

であればゼロになることが分かる。つまり、

$$ c = \frac{N-f(0)}{N} $$

であればよい。

上の図では、この関係をほぼ満たしている。

もちろんこの説明は $ f(x) = f(0) \cdot c^x $ であればうまくいくと言っているにすぎず、また、全員同じ財産から始めてそのような分布に近づいていくことも証明していない。

## ランダムに人を選び、資産を×÷ (1 + r) する

結果は対数正規分布になる。
<img src='image/dist7.png'>

In [803]:
N = 10000 # 人の数
r = 0.005
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(50000000):
    # ランダムに人を選ぶ
    x = random.randrange(N)
    # コイン投げ
    zero_one = random.randrange(2)
    if zero_one == 0:
        # その人の資産を 1 + r 倍する
        w[x] *= (1 + r)
    else:
        # その人の資産を 1 + r で割る
        w[x] /= (1 + r)
    num_iter += 1

plt.hist(w, bins=100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

掛けるか割るかを決めるためのコイン投げにおいて、表を +1、裏を -1 として合計を取ると、それは二項分布≒正規分布となる。
初期値に $1+r$ を掛ける回数（割る場合はマイナスと考える）が正規分布に従うとき、値の対数を取るとそれは正規分布となるので、元の値は（「対数正規分布」の定義により）対数正規分布となる。

## ランダムに人を選び、資産を 1 + r 倍する

これは「ランダムに人を選び、資産を×÷ (1 + r) する」の変形バージョンで、表を +1、裏を 0（なにもしない）と考えたときに相当する。
結果はやはり対数正規分布になる。
<img src='image/dist4.png'>

In [ ]:
N = 10000 # 人の数
r = 0.005
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(10000000):
    # ランダムに人を選ぶ
    x = random.randrange(N)
    # その人の資産を 1 + r 倍する
    w[x] *= (1 + r)
    num_iter += 1

plt.hist(w, bins=300)
plt.xscale("log")
x = np.arange(80, 301, 5)
plt.plot(x, 0.34 * log_normal_distribution(x, 4.96, 0.023), label='対数正規分布')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

## 資産に比例した確率でランダムに 1 与える

結果は対数正規分布になる。
<img src='image/dist3.png'>

In [213]:
def log_normal_distribution(x, mu, sigma_2):
    r = np.log(x) - mu
    return ((1.0 / math.sqrt(2.0 * math.pi * sigma_2)) * x) * np.exp(-0.5 / sigma_2 * (r * r))

In [ ]:
N = 10000 # 人の数
w0 = 10 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

range_N = range(N)
for i in range(700000):
    # 資産に比例した確率で受け取る人を選ぶ
    x = random.choices(range_N, weights=w)[0]
    # 1 与える。
    w[x] += 1
    num_iter += 1
    
plt.hist(w, bins=199)
x = np.arange(10, 201, 2)
plt.plot(x, 1.35 * log_normal_distribution(x+10, 4.36, 0.06), label='対数正規分布')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

理由は不明。分かる方がおられたらご一報いただけると幸いです。


## 資産に比例した確率で人を選び、資産を 1 + r 倍する

結果はやはり対数正規分布になる。
<img src='image/dist5.png'>

In [811]:
N = 10000 # 人の数
r = 0.01
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0
range_N = range(N)

# 資産に比例した確率で人を選び、資産を 1 + r 倍する
for i in range(500000):
    # 資産に比例した確率で人を選ぶ
    x = random.choices(range_N, weights=w)[0]
    w[x] *= (1 + r)
    num_iter += 1

plt.hist(w, bins=100)
#plt.xscale("log")
#x = np.arange(80, 301, 5)
#plt.plot(x, 0.34 * log_normal_distribution(x, 4.96, 0.023), label='対数正規分布')
plt.xlabel("財産")
plt.ylabel("人数")
#plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

## ランダムな取り引き。富が少ない方は 1 + r で割り、多い方は掛ける。

結果はべき分布になる。（以下のグラフは x，y の両方が対数目盛り）
<img src='image/dist6.png'>

In [900]:
N = 10000 # 人の数
r = 0.001
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(20000000):
    # ランダムに x, y を選ぶ。
    x, y = tuple(random.sample(range(N), k=2))
    # x の財産が少なくなるよう、必要なら入れ替える
    if w[x] > w[y]:
        x, y = y, x
    w[x] /= (1 + r) # x の財産を 1+r で割る
    w[y] *= (1 + r) # y の財産に 1+r 掛ける
    num_iter += 1

plt.hist(w, bins=500)
plt.xscale("log")
plt.yscale("log")
plt.xlim(0.1, 100)
x = np.arange(0.1, 101, 1)
plt.plot(x, 80 * np.power(x, -1.04), label='y=80 * x^-1.04')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

理由は不明。分かる方がおられたらご一報いただけると幸いです。